# Tensorflow Lite Model Deployment
#  Part 1: Make a TensorFlow model, then convert to: model.tflite

This is a generic template notebook for making a tensorflow model.
The model-creation-process is not fully reflect here. The purpose of this tool is to export a saved tensorflow model file and then to convert that file to a tflite formatted model. 

It is recommended that you create full documentation for developing your model. Once you have selected the design for your model, then you are ready to use this tool. 

Docs for a more complete workfl0w report for ML model creation here:


More Tensor Flow Reading and Examples

https://colab.research.google.com/github/tensorflow/docs/blob/snapshot-keras/site/en/guide/keras/train_and_evaluate.ipynb


## For Notebook installs into env

In [ ]:
#!pip install --upgrade pip
#!pip install numpy 
#!pip install matplotlib
#!pip install tensorflow

## import

In [3]:

# import and alias python libraries and packages
import tensorflow as tf

import pathlib
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input

from sklearn.model_selection import train_test_split


# Import Data

In [ ]:
# get your data


# Split Data Test/Train/Validate
## (Various Versions Below)

## Super Simple Version


In [ ]:
# Create a simple Keras model.
x = [-1, 0, 1, 2, 3, 4]
y = [-3, -1, 1, 3, 5, 7]



# Iris Dataset Version

In [4]:
# DEMO ONLY Using iris data

from sklearn.datasets import load_iris

iris = load_iris()

X = iris.data
y = iris.target

#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)

# SKlearn Version
## use scikit-learn for your split

In [ ]:
X = YOUR_X_DATA
y = YOUR_y_DATA

#Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)

# inspection
See what you have:

In [ ]:
X_train

In [ ]:
y_train

# Preserve All X and y for final training:

In [ ]:
X_all = X
y_all = y

## Select Type of Model

In [5]:
# EXAMPLE

# note: for this model, update X input shape

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[4])
])
model.compile(optimizer='sgd', loss='mean_squared_error')


## .fit
## Train The Model

In [ ]:
model.fit(X_train, y_train, epochs=20, verbose=1)

## Evaluate The Model

https://www.tensorflow.org/guide/keras/train_and_evaluate

https://colab.research.google.com/github/tensorflow/docs/blob/snapshot-keras/site/en/guide/keras/train_and_evaluate.ipynb#scrollTo=69f524a93f9d

In [7]:
# Evaluate the model on the test data using `evaluate`

# Use Tensorflow evalutate tool:
results = model.evaluate(X_test, y_test, batch_size=128)

print("test loss, test acc:", results, "\n")

# Generate predictions 
# on new data using `predict`
print("Generate predictions to test output shape:")
predictions = model.predict(X_test[:3])

# Test Shape
print("predictions shape:", predictions.shape)

1/1 [==============================] - 0s 108ms/step - loss: 0.1124
test loss, test acc: 0.11241965740919113 

Generate predictions to test output shape:
predictions shape: (3, 1)


# For proper testing, save a portable version and test from that:

In [12]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Save your model in .h5 format (one file, not a directory)
model.save('model.h5')

# Load the .h5 model file and test it
model = load_model('model.h5')

## Run the same test as above, but this will use your saved model.

In [13]:
# Use Tensorflow evalutate tool:
results = model.evaluate(X_test, y_test, batch_size=128)

print("test loss, test acc:", results, "\n")

# Generate predictions 
# on new data using `predict`
print("Generate predictions to test output shape:")
predictions = model.predict(X_test[:3])

# Test Shape
print("predictions shape:", predictions.shape)

1/1 [==============================] - 0s 135ms/step - loss: 0.1124
test loss, test acc: 0.11241965740919113 

Generate predictions to test output shape:
predictions shape: (3, 1)


# Confusion Matrix

(work in progress...)

https://www.tensorflow.org/api_docs/python/tf/math/confusion_matrix 

https://www.geeksforgeeks.org/python-tensorflow-math-confusion_matrix/

In [ ]:
# # importing the library
# import tensorflow as tf

# # Initializing the input tensor
# labels = tf.constant([1,3,4],dtype = tf.int32)
# predictions = tf.constant([1,2,3],dtype = tf.int32)

# # Printing the input tensor
# print('labels: ',labels)
# print('Predictions: ',predictions)

# # Evaluating confusion matrix
# confusion_matrix = tf.math.confusion_matrix(labels,predictions)

# # Printing the result
# print('Confusion_matrix: ',confusion_matrix)


# Re-Train on whole data set before deploying

In [ ]:
model.fit(X_all, y_all, epochs=200, verbose=1)

## Save Tensorflow Model (NOT yet .tflite)

In [11]:
# makde a .h5 version of the model
model.save('model.h5')

# inspect
!ls

In [8]:
TF_model_directory = 'saved_model/'
tf.saved_model.save(model, TF_model_directory)

# inspect
!ls

INFO:tensorflow:Assets written to: saved_model/assets
sample_data  saved_model


## Make .ftlite Model (Convert Tensorflow Model to .tflite)

# Make 'model.tflite'

Note: the suffix is fixed, but you can change
the first part of the name,
but the sample code here assumes "model.tflite"


```
YOUR_CHOICE_OF_NAME.tflite
```

In [9]:
#########################################
# Make Your TFlite Version of your Model
#########################################
# See: https://www.tensorflow.org/lite/convert/

# save the TF model as full directory
TF_model_directory = 'saved_model/'
tf.saved_model.save(model, TF_model_directory)

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model( TF_model_directory ) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model as a single TFlite file: model.tflite
# you can put the name, but the filetype must be .tflite
# YOUR_NAME_CHOICE.tflite
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# inspect
!ls

## Now your model.tflite file is ready to be deployed.

## (Optional) Test .tflite model using tensorflow 

### Note: this is a kind of emulator in TF, this is NOT the deployment code to use in TFlite. This is just a model-test.


In [10]:

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


# Make A Prediction (short version)

# X: input data
X_raw_input = [[0.37094948,0.37094948,0.37094948,0.37094948]]
# convert raw input number to an numpy array
# this is required for TFlite??
input_data = np.asarray(X_raw_input, dtype=np.float32)
    
# y: using model, producing y from X
interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

# inspection
print("y =", tflite_results)


y = [[0.19107173]]


# Helper  Code: Convert .h5 to .tflite

https://github.com/lineality/deploy_TFlite_tensorflow_lite_in_aws_lambda_function

In [ ]:
"""
TF TFlite Convert from model.h5 to model.tflite

Step 1: Put your model.h5 file into the directory

Step 2: Convert .h5 to .tflite

Step 3: Download / Save your models and use them
(e.g. download from google colab so the file you made doesn't disappear. 
Right click on model.ftlite in the file explorere and select 'download' )
"""


import tensorflow as tf
from tensorflow.keras.models import load_model

#######
# Load
#######

# Load the .h5 model file
model = load_model('model.h5')

# Save the TF model as directory
TF_model_directory = 'saved_model/'
tf.saved_model.save(model, TF_model_directory)

####################################
# Make TFlite Version of your Model
####################################

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model( TF_model_directory ) # path to the SavedModel directory
tflite_model = converter.convert()

####################################
# Save TFlite Version of your Model
####################################

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# inspect
!ls